
# Lab 4: Using Access Security Settings and Policies 
## Applying Access Control Expressions (ACE) to Volumes Using ACE's and Global Security Policies 

## Security on Ezmeral Data Fabric.

Security on the Ezmearl Data Fabric is applied at the Volume which can be applied to:

- Directories and files
- Tables (if you have the full features Ezmeral Data Fabric)
- Streams (if you have the full features Ezmeral Data Fabric)

Security can be applied in two ways:

- On-demand per data type by creating an Access Control Experssion (ACE) and then applying that ACE as a security tag to the volume.
- Or through a local or global policy management which creates a catalog of security tags that can then be applied to any data type.

Below is an example we will be running through for this lab:

<p align="center">
  <img src="Pictures/edf-sec-ace-example.png" width="800" height="600" alt="Data Fabric Snapshots">                                                    
  
</p>

For more information on security features within the Ezmeral Data Fabric please see [Managing Security Policies](https://docs.datafabric.hpe.com/62/SecurityGuide/ManagingSecurityPolicies.html)

In [1]:
%login 16.31.86.198

echo "We are now connected to the appliance"

#Variables used by scripts
#password="qcpx0k4u" # your Data Fabric login credentials - password - 
#apiserver_host="16.31.85.156" # API Server for the Data Fabric Cluster
#ad_group="data-fabric-users" # LDAP Domain Group for Data Fabric users
#vol_name="vol1" # suffix for student volume name
#snapvol_name="snapvol1" # suffix for student snap volume name
#cluster_name="mapr-cluster2.etc.fr.comm.hpecorp.net" # Data Fabric Cluster Name
#src_vol_name="src-mirrvol1" # suffix for student source mirror volume name
#dst_vol_name="dst-mirrvol1"  # suffix for student destination mirror volume name
#secvol_name="sec-vol1"  # suffix for student volume name used for security comparison / user access 
#sec_policy="student1222-policy" #security policy to allow r/w access to volumes

echo "your operation context is:" student1222 "with Password:" qcpx0k4u
echo "----"
echo "your test user for applying security policy context is:" sec-student1222 "with Password:" qcpx0k4u

[ssh] Login to 16.31.86.198...
[ssh] host=16.31.86.198 hostname=16.31.86.198 other_conf={'user': None, 'port': None, 'keyfile': None, 'load_system_ssh_config': False, 'missing_host_policy': <paramiko.client.WarningPolicy object at 0x7f48f02de490>}
[ssh] Successfully logged in.
[ssh] host = 16.31.86.198, cwd = /student/student1222
We are now connected to the appliance
your operation context is: student1222 with Password: qcpx0k4u
----
your test user for applying security policy context is: sec-student1222 with Password: qcpx0k4u


## 1. Create a volume that we can apply security policies.

#### Background:

You have been given a user that is allowed to create and modify volumes on the cluster and another user to test your security policies. Data Fabric volumes can be mounted so they are accessible from NFS, HDFS or POSIX commands (like the bash kernel shell in these labs). For security reasons, you can only mount volumes in the /user directory.

In this lab example we will:

1.	Created a new volume
2.	Tested writing and reading data to it from multiple users.
3.	Applied a ‘read only’ Access Control Expression to the volume.
4.	Tested writing and reading data to it from the sec-studentXXXX user.
5.	Created a security policy with read and write Access Control Expression’s.
6.	Apply the security policy to the volume.
7.	Tested writing and reading data to it from the sec-studentXXXX user.

To understand how an ACE works on a volume with files and directories first let's create a volume that all users can write files to:


In [2]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/create?name='$username-$secvol_name'&path=/user/'$username-$secvol_name'&rootdirgroup=Data-Fabric-Users&rootdirperms=777' --user $username:$password

[ssh] host = 16.31.86.198, cwd = /student/student1222
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   178  100   178    0     0     69      0  0:00:02  0:00:02 --:--:--    70
{"timestamp":1624511635061,"timeofday":"2021-06-24 07:13:55.061 GMT+0200 AM","status":"OK","total":0,"data":[],"messages":["Successfully created volume: 'student1222-sec-vol1'"]}

## 2. Test that both users can read/write data to the created volume

- We will list out the files and create/append data to a file as the username:

You are logging in as the user:
<p align="center">
  <img src="Pictures/studentxxxx.png" alt="studentxxx-Admin">                                                    
  
</p>

In [3]:
source ~/.bashrc
echo "##Verify which user we are curently logged in as.##"
echo $username

[ssh] host = 16.31.86.198, cwd = /student/student1222
##Verify which user we are curently logged in as.##
student1222


In [4]:
source ~/.bashrc
echo "##Create a new file on our security volume.##"
echo "$username-$secvol_name security volume update $(date)" >> /mapr/$cluster_name/user/$username-$secvol_name/$username-security-volume-test1
echo "##List out the volume directory.##"
ls /mapr/$cluster_name/user/$username-$secvol_name
echo "##Verify the file contents we just wrote.##"
cat /mapr/$cluster_name/user/$username-$secvol_name/$username-security-volume-test1

[ssh] host = 16.31.86.198, cwd = /student/student1222
##Create a new file on our security volume.##
##List out the volume directory.##
student1222-security-volume-test1
##Verify the file contents we just wrote.##
student1222-sec-vol1 security volume update Thu Jun 24 07:13:22 CEST 2021


In [5]:
%logout

[ssh] Closing existing connection.
[ssh] Successfully logged out.


- Next we will verify that another user can list out the files and create/append data to a file as the username:

You are logging in as the user:
<p align="center">
  <img src="Pictures/sec-studentxxxx.png" alt="studentxxx-Dev">                                                    
  
</p>

In [6]:
%login sec-appliance

cat > $HOME/.bashrc << EOF
username="student1222" # your Data Fabric login credentials - username  
password="qcpx0k4u" # your Data Fabric login credentials - password - 
sec_username="sec-student1222" # your Data Fabric login credentials - sec username  
sec_password="stuDISCO2020" # your Data Fabric login credentials - sec password - 
apiserver_host="16.31.85.156" # API Server for the Data Fabric Cluster
ad_group="data-fabric-users" # LDAP Domain Group for Data Fabric users
vol_name="vol1" # suffix for student volume name
snapvol_name="snapvol1" # suffix for student snap volume name
cluster_name="mapr-cluster2.etc.fr.comm.hpecorp.net" # Data Fabric Cluster Name
src_mirrvol1="src-mirrvol1" # suffix for student source mirror volume name
dst_mirrvol1="dst-mirrvol1"  # suffix for student destination mirror volume name
secvol_name="sec-vol1"  # suffix for student volume name used for security comparison / user access 

EOF


source ~/.bashrc
echo "login as sec-student1222"
id
echo $sec_username

[ssh] Login to sec-appliance...
[ssh] host=sec-appliance hostname=16.31.86.198 other_conf={'user': 'sec-student1222', 'port': None, 'keyfile': None, 'load_system_ssh_config': False, 'missing_host_policy': <paramiko.client.WarningPolicy object at 0x7f48f02de550>}
[ssh] Successfully logged in.
[ssh] host = sec-appliance, cwd = /student/sec-student1222
login as sec-student1222
uid=15222(sec-student1222) gid=14222(sec-student1222) groups=14222(sec-student1222),36000(Data-Fabric-Users) context=unconfined_u:unconfined_r:unconfined_t:s0-s0:c0.c1023
sec-student1222


In [7]:
source ~/.bashrc
echo "##Create a new file as a different user on our security volume.##"
echo "$sec_username-$secvol_name security volume sec user update $(date)" >> /mapr/$cluster_name/user/$username-$secvol_name/$username-security-volume-secuser-test1
echo "##List out the volume directory.##"
ls /mapr/$cluster_name/user/$username-$secvol_name/
echo "##Verify the file contents we just wrote as a new user.##"
cat /mapr/$cluster_name/user/$username-$secvol_name/$username-security-volume-secuser-test1


[ssh] host = sec-appliance, cwd = /student/sec-student1222
##Create a new file as a different user on our security volume.##
##List out the volume directory.##
student1222-security-volume-secuser-test1
student1222-security-volume-test1
##Verify the file contents we just wrote as a new user.##
sec-student1222-sec-vol1 security volume sec user update Thu Jun 24 07:13:27 CEST 2021


In [8]:
%logout

[ssh] Closing existing connection.
[ssh] Successfully logged out.


## 3. Modify a volume and apply a read Access Control Expression (ACE)

A valid user or group must be present on your cluster or in your directory services.  In this case we will use the following user to apply a read only ACE to the volume:

You are logging in as the user:
<p align="center">
  <img src="Pictures/studentxxxx.png" alt="studentxxx-Admin">                                                    
  
</p>


In [9]:
%login 16.31.86.198

echo "We are now connected to the appliance"

#Variables used by scripts
#password="qcpx0k4u" # your Data Fabric login credentials - password - 
#apiserver_host="16.31.85.156" # API Server for the Data Fabric Cluster
#ad_group="data-fabric-users" # LDAP Domain Group for Data Fabric users
#vol_name="vol1" # suffix for student volume name
#snapvol_name="snapvol1" # suffix for student snap volume name
#cluster_name="mapr-cluster2.etc.fr.comm.hpecorp.net" # Data Fabric Cluster Name
#src_vol_name="src-mirrvol1" # suffix for student source mirror volume name
#dst_vol_name="dst-mirrvol1"  # suffix for student destination mirror volume name
#secvol_name="sec-vol1"  # suffix for student volume name used for security comparison / user access 
#sec_policy="student1222-policy" #security policy to allow r/w access to volumes

echo "your operation context is:" student1222 "with Password:" qcpx0k4u

[ssh] Login to 16.31.86.198...
[ssh] host=16.31.86.198 hostname=16.31.86.198 other_conf={'user': None, 'port': None, 'keyfile': None, 'load_system_ssh_config': False, 'missing_host_policy': <paramiko.client.WarningPolicy object at 0x7f48f0239a60>}
[ssh] Successfully logged in.
[ssh] host = 16.31.86.198, cwd = /student/student1222
We are now connected to the appliance
your operation context is: student1222 with Password: qcpx0k4u


In [10]:
source ~/.bashrc
echo "## We will now apply an ACE to the security volume, in this example the new user will only be allowed 'Read Only' access. ##"
curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/modify?name='$username-$secvol_name'&writeAce=u%3A'$username'&readAce=u%3A'$sec_username'' --user $username:$password

[ssh] host = 16.31.86.198, cwd = /student/student1222
## We will now apply an ACE to the security volume, in this example the new user will only be allowed 'Read Only' access. ##
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   111  100   111    0     0    352      0 --:--:-- --:--:-- --:--:--   353
{"timestamp":1624511647096,"timeofday":"2021-06-24 07:14:07.096 GMT+0200 AM","status":"OK","total":0,"data":[]}

In [11]:
%logout

[ssh] Closing existing connection.
[ssh] Successfully logged out.


## 4. Verify the security policy change.

We will now verify that the previous step we applied only has read-only access and is unable to write any data to the directory or file for the user:

You are logging in as the user:
<p align="center">
  <img src="Pictures/sec-studentxxxx.png" alt="studentxxx-Dev">                                                    
  
</p>

In [12]:
%login sec-appliance

cat > $HOME/.bashrc << EOF
username="student1222" # your Data Fabric login credentials - username  
password="qcpx0k4u" # your Data Fabric login credentials - password - 
sec_username="sec-student1222" # your Data Fabric login credentials - sec username  
sec_password="stuDISCO2020" # your Data Fabric login credentials - sec password - 
apiserver_host="16.31.85.156" # API Server for the Data Fabric Cluster
ad_group="data-fabric-users" # LDAP Domain Group for Data Fabric users
vol_name="vol1" # suffix for student volume name
snapvol_name="snapvol1" # suffix for student snap volume name
cluster_name="mapr-cluster2.etc.fr.comm.hpecorp.net" # Data Fabric Cluster Name
src_mirrvol1="src-mirrvol1" # suffix for student source mirror volume name
dst_mirrvol1="dst-mirrvol1"  # suffix for student destination mirror volume name
secvol_name="sec-vol1"  # suffix for student volume name used for security comparison / user access 
sec_policy="student1222-policy" #security policy to allow r/w access to volumes
EOF

echo "login as sec-student1222"
id

[ssh] Login to sec-appliance...
[ssh] host=sec-appliance hostname=16.31.86.198 other_conf={'user': 'sec-student1222', 'port': None, 'keyfile': None, 'load_system_ssh_config': False, 'missing_host_policy': <paramiko.client.WarningPolicy object at 0x7f48f01f8040>}
[ssh] Successfully logged in.
[ssh] host = sec-appliance, cwd = /student/sec-student1222
login as sec-student1222
uid=15222(sec-student1222) gid=14222(sec-student1222) groups=14222(sec-student1222),36000(Data-Fabric-Users) context=unconfined_u:unconfined_r:unconfined_t:s0-s0:c0.c1023


In [13]:
source ~/.bashrc
echo "## Make an update to the file as the new user. ##"
echo "$sec_username-$secvol_name security volume sec user update $(date)" >> /mapr/$cluster_name/user/$username-$secvol_name/$username-security-volume-secuser-test1
echo "## List out the volume diretory to the files. ##"
ls /mapr/$cluster_name/user/$username-$secvol_name/
echo "## Verify the update to the file as the new user. ##"
cat /mapr/$cluster_name/user/$username-$secvol_name/$username-security-volume-secuser-test1
echo "##"
echo "## You should have recieved a 'Pemission denied', this is expected as the new user only has 'Read only' access now to the volume. ##"


[ssh] host = sec-appliance, cwd = /student/sec-student1222
## Make an update to the file as the new user. ##
## List out the volume diretory to the files. ##
/usr/bin/bash: line 3: /mapr/mapr-cluster2.etc.fr.comm.hpecorp.net/user/student1222-sec-vol1/student1222-security-volume-secuser-test1: Permission denied
student1222-security-volume-secuser-test1
student1222-security-volume-test1
## Verify the update to the file as the new user. ##
sec-student1222-sec-vol1 security volume sec user update Thu Jun 24 07:13:27 CEST 2021
##
## You should have recieved a 'Pemission denied', this is expected as the new user only has 'Read only' access now to the volume. ##


In [14]:
%logout

[ssh] Closing existing connection.
[ssh] Successfully logged out.


## 5. Creating Security Policies

#### Background

Security polices allow you to create a central policy that can then be applied to multiple data objects.  In this case a policy can have an Access Control Expression (ACE) for a set of users or groups and you then apply that policy as a "tag" to your volume.

We will first create a security policy for the authentication services group that has all of our students in ito the directory or file for the user:


You are logging in as the user:
<p align="center">
  <img src="Pictures/studentxxxx.png" alt="studentxxx-Admin">                                                    
  
</p>



In [15]:
%login 16.31.86.198

[ssh] Login to 16.31.86.198...
[ssh] host=16.31.86.198 hostname=16.31.86.198 other_conf={'user': None, 'port': None, 'keyfile': None, 'load_system_ssh_config': False, 'missing_host_policy': <paramiko.client.WarningPolicy object at 0x7f48f0270d90>}
[ssh] Successfully logged in.


In [16]:
source ~/.bashrc
echo "## Create a new security policy with write and read access and add the new user to it. ##"
curl -k -X POST 'https://'$apiserver_host':8443/rest/security/policy/create?name='$sec_policy'&allowtagging=true&accesscontrol=Armed&wiresecurityenabled=true&readaces=g%3A'$ad_group'&writeaces=u%3A'$sec_username'' --user $username:$password
echo "## Reset the individual ACE on the volume. ##"
curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/modify?name='$username-$secvol_name'&writeAce=u%3A'$sec_username'' --user $username:$password

[ssh] host = 16.31.86.198, cwd = /student/student1222
## Create a new security policy with write and read access and add the new user to it. ##
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   203  100   203    0     0    376      0 --:--:-- --:--:-- --:--:--   377
{"timestamp":1624511653267,"timeofday":"2021-06-24 07:14:13.267 GMT+0200 AM","status":"OK","total":0,"data":[],"messages":["Successfully created security policy 'student1222-secpol-43f2b52b-a550-476c'"]}## Reset the individual ACE on the volume. ##
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   111  100   111    0     0    373      0 --:--:-- --:--:-- --:--:--   375
{"timestamp":1624511653577,"timeofday":"2021-06-24 07:14:13.577 GMT+0200 AM","status":"OK","total":0,"data":[]}

Now let's apply our policy we created to the volume:

In [17]:
source ~/.bashrc
echo "## Apply the new security policy to your volume. ##"
curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/modify?name='$username-$secvol_name'&securitypolicy='$sec_policy'' --user $username:$password

[ssh] host = 16.31.86.198, cwd = /student/student1222
## Apply the new security policy to your volume. ##
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   167  100   167    0     0    636      0 --:--:-- --:--:-- --:--:--   637
{"timestamp":1624511653928,"timeofday":"2021-06-24 07:14:13.928 GMT+0200 AM","status":"ERROR","errors":[{"id":22,"desc":"non-existent or untaggable security policy"}]}

In [18]:
%logout

[ssh] Closing existing connection.
[ssh] Successfully logged out.


Now let's test that our other users can write files to the volume now:

You are logging in as the user:
<p align="center">
  <img src="Pictures/sec-studentxxxx.png" alt="studentxxx-Dev">                                                    
  
</p>

In [19]:
%login sec-appliance

[ssh] Login to sec-appliance...
[ssh] host=sec-appliance hostname=16.31.86.198 other_conf={'user': 'sec-student1222', 'port': None, 'keyfile': None, 'load_system_ssh_config': False, 'missing_host_policy': <paramiko.client.WarningPolicy object at 0x7f48f02d10d0>}
[ssh] Successfully logged in.


In [20]:
source ~/.bashrc
echo "## Make an update to the file as the new user. ##"
echo "$sec_username-$secvol_name security volume sec user update $(date)" >> /mapr/$cluster_name/user/$username-$secvol_name/$username-security-volume-secuser-test1
echo "## List out the volume diretory to the files. ##"
ls /mapr/$cluster_name/user/$username-$secvol_name/
echo "## Verify the update to the file as the new user. ##"
cat /mapr/$cluster_name/user/$username-$secvol_name/$username-security-volume-secuser-test1



[ssh] host = sec-appliance, cwd = /student/sec-student1222
## Make an update to the file as the new user. ##
## List out the volume diretory to the files. ##
/usr/bin/bash: line 3: /mapr/mapr-cluster2.etc.fr.comm.hpecorp.net/user/student1222-sec-vol1/student1222-security-volume-secuser-test1: Permission denied
student1222-security-volume-secuser-test1
student1222-security-volume-test1
## Verify the update to the file as the new user. ##
sec-student1222-sec-vol1 security volume sec user update Thu Jun 24 07:13:27 CEST 2021


In [21]:
%logout

[ssh] Closing existing connection.
[ssh] Successfully logged out.


### Let's cleanup now

 1. Remove Security Policies
 2. Remove Volumes

In [22]:
%login 16.31.86.198


[ssh] Login to 16.31.86.198...
[ssh] host=16.31.86.198 hostname=16.31.86.198 other_conf={'user': None, 'port': None, 'keyfile': None, 'load_system_ssh_config': False, 'missing_host_policy': <paramiko.client.WarningPolicy object at 0x7f48f02d1820>}
[ssh] Successfully logged in.


In [23]:
source ~/.bashrc

#Remove Volume
curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/remove?name='$username-$secvol_name'' --user $username:$password

[ssh] host = 16.31.86.198, cwd = /student/student1222
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   111  100   111    0     0     47      0  0:00:02  0:00:02 --:--:--    47
{"timestamp":1624511665326,"timeofday":"2021-06-24 07:14:25.326 GMT+0200 AM","status":"OK","total":0,"data":[]}

In [24]:
%logout 

[ssh] Closing existing connection.
[ssh] Successfully logged out.


For more information on creating security policies please see [Creating Security Policies](https://docs.datafabric.hpe.com/62/SecurityGuide/CreatingSecurityPolicies.html) and [Managing File and Directory ACE's](https://docs.datafabric.hpe.com/62/SecurityGuide/FileDirACE.html)

## Using Access Security Settings and Policies Summary

In this example we:

1.	Created a new volume
2.	Tested writing and reading data to it from multiple users.
3.	Applied a ‘read only’ Access Control Expression to the volume.
4.	Tested writing and reading data to it from the sec-studentXXXX user.
5.	Created a security policy with read and write Access Control Expression’s.
6.	Applied the security policy to the volume.
7.	Tested writing and reading data to it from the sec-studentXXXX user.


# Time to wrap up
You can now move to the conclusion of this workshop !

* [Conclusion](5-WKSHP-Conclusion.ipynb)